# Strange Signals


![Image](set.jpeg)



In [ ]:
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_md-3.5.0/en_core_web_md-3.5.0-py3-none-any.whl


In [ ]:
import pandas as pd
import seaborn as sns
sns.set()
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from IPython.display import IFrame
import random
import sympy
from collections import Counter
import spacy
nlp = spacy.load('en_core_web_md')
import sklearn
from sklearn.decomposition import PCA
from sklearn.cluster import AgglomerativeClustering
import plotly.express as px
from sklearn.cluster import KMeans
from sklearn.cluster import AffinityPropagation
from scipy.spatial import distance
import matplotlib.pyplot as plt

# The problem: How might we decode an alien signal?

![The Wow! Signal](wow_signal.png)

# The difference between syntax and semantics

The first step in decoding any alien language comes with distiguishing beween the rules that allow tokens to be combined, and what these tokens mean.

1. Syntax concerns the formal rules that allow words or tokens to be put together in legitimate ways.
2. Semantics concerns the meaning of words and word sequences.

Take the sentence: 

## "April is the cruellest month". 

The syntax of this sentence is determined by the gramatical relations between its parts:

![Syntax](eliot.png)

The semantic structure of the sentence is determined by the meaning of the words and how they map onto the world.

![Semantics](april.png)

If we are to decode an alien language, we need to analyse both its syntax and its semantics. 

## How will we procede?

### Syntax

* We will use a concept called entropy to help us figure out the 'alphabet' of the alien language. This will then allow us to determine its words.

### Semantics

* We will use AI methods to figure out the semantics of the language, and see if there is any plausible mapping between the ways concepts are combined in human languages and the alien language.

# 1. Syntax

# Entropy and natural language processing (NLP)

Entropy is a measure of unpredictability; the more unpredictable a system is, the higher its entropy. Though originally formulated in the context of thermodynamics, Claude Shannon extended the concept to information in his landmark work [A Mathematical Theory of Communication](https://people.math.harvard.edu/~ctm/home/text/others/shannon/entropy/entropy.pdf). Because it offers a measure of unpredictability, entropy is of crucual important in NLP, machine learning, and AI. 

This is the formula for entropy, where $H$ is the entropy measure and $X$ is a discrete probability distribution over $n$ states of a system:

$$X = (x_1, x_2, x_3, ... x_n)$$

$$H(X) = -\sum_{x \in X} p(x)\log_{2} p(x)$$

But what does this formula mean? Let's investigate.

## Surprise

The first step in understanding entropy comes with defining the notion of *surprise* (sometimes called *surprisal*, *information*, or *self-information*). When is an event surprising? When it's unlikely but happens anyway. Therefore, surprise is inveresely proportional to probability: high probability events have low surprise (we expect them to occur) while low probability events have high surprise (we *don't* expect them to occur). 

Let's take an example. What would surprise us most if it fell from the sky?


In [ ]:
x = ['frogs', 'ash', 'snow', 'rain']
y = [0.01, 0.1, 0.3, 0.59]

sns.barplot(x = x, y = y)
plt.ylim(0, 1) 
plt.ylabel('probability')
plt.xlabel('falling items')
plt.title("What's falling from the sky?")

### The logarithm function as a measure of surprise

How might we represent this mathematically? First, note that every probability must take a value between $0$ and $1$, where the sum of all values in a distribution equals 1. This means that we want a formmula for surprise that turns small probability values into large surprise values, and vice versa. Usefully, the $-log(x)$ function does this between the values of $0$ and $1$:

In [ ]:

x = np.arange(0,1, 0.0001) # Gets the numbers between 0 and 1 in increments of 0.0001 using the numpy (np) library 
y = [-np.log2(i) for i in x] # Gets the log base 2 of these numbers

sns.lineplot(x = x, y = y) # Plots y against x
plt.ylabel('surprise (bits)')
plt.xlabel('probability')
plt.title('Surprise over probability')

As can be seen, as probability increases surprise decreases––which is exactly what we want for our measure of surprise. How can we use this to characterise the unpredictability (i.e. the entropy) of an entire system? This is done by multiplying the probability of every state of a system by its surprise, and adding the results together:

$$X = (x_1, x_2, ... , x_n)$$

$$H(X) = p(x_1)\times -\log_{2}p(x_1) + p(x_2)\times -\log_{2}p(x_2) + ...+ p(x_n)\times -\log_{2}p(x_n)$$

Notice that when we represent this using the summation operator ... we get the formula for entropy! 

$$H(X) = -\sum_{x \in X} p(x)\log_{2} p(x)$$

So entropy, in this sense, can be defined as the expected value of surprise across all states or outcomes of a system. That is, it's the 'average' amount of surprise across the system. 



## Example 1––A music recommendation system
| Title                                    | Genre   | Frequency |
|------------------------------------------|---------|-----------|
| Blinding Lights - The Weeknd             | Pop     | 25        |
| Bad Guy - Billie Eilish                  | Pop     | 40        |
| Shake It Off - Taylor Swift              | Pop     | 36        |
| Uptown Funk - Mark Ronson ft. Bruno Mars | Pop     | 42        |
| Happy - Pharrell Williams                | Pop     | 29        |
| Roar - Katy Perry                        | Pop     | 34        |
| Sorry - Justin Bieber                    | Pop     | 33        |
| Rolling in the Deep - Adele              | Pop     | 27        |
| Shape of You - Ed Sheeran                | Pop     | 38        |
| Levitating - Dua Lipa                    | Pop     | 30        |
| Bohemian Rhapsody - Queen                | Rock    | 78        |
| Stairway to Heaven - Led Zeppelin        | Rock    | 65        |
| Hotel California - Eagles                | Rock    | 82        |
| Back in Black - AC/DC                    | Rock    | 90        |
| Sweet Child O' Mine - Guns N' Roses      | Rock    | 74        |
| Smells Like Teen Spirit - Nirvana        | Rock    | 68        |
| Imagine - John Lennon                    | Rock    | 85        |
| Free Fallin' - Tom Petty                 | Rock    | 62        |
| Livin' on a Prayer - Bon Jovi            | Rock    | 88        |
| Thunderstruck - AC/DC                    | Rock    | 71        |
| Jolene - Dolly Parton                    | Country | 18        |
| Take Me Home, Country Roads - John Denver| Country | 22        |
| The Gambler - Kenny Rogers               | Country | 20        |
| Ring of Fire - Johnny Cash               | Country | 25        |
| Tennessee Whiskey - Chris Stapleton      | Country | 17        |
| Before He Cheats - Carrie Underwood      | Country | 19        |
| Take Five - Dave Brubeck                 | Jazz    | 6         |
| So What - Miles Davis                    | Jazz    | 8         |
| What a Wonderful World - Louis Armstrong | Jazz    | 9         |
| Feeling Good - Nina Simone               | Jazz    | 4         |


#### Make our data python compatible by creating a dataframe

In [ ]:
songs = [
    "Blinding Lights - The Weeknd",
    "Bad Guy - Billie Eilish",
    "Shake It Off - Taylor Swift",
    "Uptown Funk - Mark Ronson ft. Bruno Mars",
    "Happy - Pharrell Williams",
    "Roar - Katy Perry",
    "Sorry - Justin Bieber",
    "Rolling in the Deep - Adele",
    "Shape of You - Ed Sheeran",
    "Levitating - Dua Lipa",
    "Bohemian Rhapsody - Queen",
    "Stairway to Heaven - Led Zeppelin",
    "Hotel California - Eagles",
    "Back in Black - AC/DC",
    "Sweet Child O' Mine - Guns N' Roses",
    "Smells Like Teen Spirit - Nirvana",
    "Imagine - John Lennon",
    "Free Fallin' - Tom Petty",
    "Livin' on a Prayer - Bon Jovi",
    "Thunderstruck - AC/DC",
    "Jolene - Dolly Parton",
    "Take Me Home, Country Roads - John Denver",
    "The Gambler - Kenny Rogers",
    "Ring of Fire - Johnny Cash",
    "Tennessee Whiskey - Chris Stapleton",
    "Before He Cheats - Carrie Underwood",
    "Take Five - Dave Brubeck",
    "So What - Miles Davis",
    "What a Wonderful World - Louis Armstrong",
    "Feeling Good - Nina Simone"
]

frequencies = [
    25, 40, 36, 42, 29, 34, 33, 27, 38, 30,
    78, 65, 82, 90, 74, 68, 85, 62, 88, 71,
    18, 22, 20, 25, 17, 19,
    6, 8, 9, 4
]

genres = [
    "Pop", "Pop", "Pop", "Pop", "Pop", "Pop", "Pop", "Pop", "Pop", "Pop",
    "Rock", "Rock", "Rock", "Rock", "Rock", "Rock", "Rock", "Rock", "Rock", "Rock",
    "Country", "Country", "Country", "Country", "Country", "Country",
    "Jazz", "Jazz", "Jazz", "Jazz"
]

playlist = pd.DataFrame()
playlist['song'] = songs
playlist['play_count'] = frequencies
playlist['genre'] = genres
playlist['probability'] = [i/playlist['play_count'].sum() for i in playlist['play_count']]


In [ ]:
songs_prob = playlist['probability']
genres_prob = playlist.groupby('genre').sum()['probability']

songs = sp.stats.entropy(songs_prob, base = 2) # Gets the entropy using log base 2 
genres = sp.stats.entropy(genres_prob, base = 2)

print("The entropy of the listener's taste based on songs is {} bits.".format(songs),"The entropy of the listener's taste based on genre is {} bits.".format(genres))

In [ ]:
genres = playlist.groupby('genre').sum()['probability']

In [ ]:
letter_frequencies =  {'E': 0.12003601080324099, 'T': 0.09102730819245775, 'A': 0.08122436731019306, \
                       'O': 0.07682304691407423, 'I': 0.0731219365809743, 'N': 0.06952085625687708, \
                           'S': 0.06281884565369612, 'R': 0.06021806541962589, 'H': 0.05921776532959889, \
                               'D': 0.04321296388916676, 'L': 0.03981194358307493, 'U': 0.028808642592777836, \
                                   'C': 0.027108132439731925, 'M': 0.026107832349704915, 'F': 0.02300690207062119, \
                                       'Y': 0.021106331899569872, 'W': 0.02090627188156447, 'G': 0.020306091827548264, \
                                           'P': 0.01820546163849155, 'B': 0.014904471341402423, 'V': 0.011103330999299792,\
                                               'K': 0.006902070621186356, 'X': 0.0017005101530459142, 'Q': 0.0011003300990297092, \
                                                   'J': 0.0010003000900270084, 'Z': 0.0007002100630189059}

In [ ]:
sns.lineplot(y = [i for i in letter_frequencies.keys()], x = [i for i in letter_frequencies.values()])

In [ ]:
entropy_alphabet = sp.stats.entropy([i for i in letter_frequencies.values()], base = 2)
                                     
random_freq = [1] * 26
random_freq = [1/26 for i in random_freq]
                                     
entropy_rand = sp.stats.entropy(random_freq, base = 2)
                                     
print("The entropy of the English alphabet is {} bits.".format(entropy_alphabet),"The entropy of a system of 26 equiprobable symbols is {} bits.".format(entropy_rand))

# Our alien message

What's the most basic way a message can be encoded? It's as a series of 0's and 1's. This is because any transmitting device can transmit or not transmit. The transmissions give us the 1's; the gaps between them give us the 0's. 

So let's imagine we receive the series of 0's and 1's below. 

In [ ]:

text = '011101110110010100000000011000110110111101101101011001010000000001110110011001010111001001111001000000000110001001110010011010010110010101100110011011000111100100000000011101000110111100000000011101000110100001101001011100110000000001110000011011000110000101100011011001010000000001100001011011100110010000000000011101110110000101110100011000110110100000000000011110010110111101110101011100100000000001110011011101000110000101110010000000000110011001110010011011110110110100000000011000010110011001100001011100100000000001110011011011110000000001110100011010000110000101110100000000000111011101100101000000000110110101100001011110010000000001110011011001010110010100000000011110010110111101110101000000000110100101101110000000000111010001101000011001010000000001101100011010010110011101101000011101000000000001101111011001100000000001111001011011110111010101110010000000000110001001100101011010010110111001100111'

In [ ]:
print(text)

In [ ]:
len(text)

## Now, let's generate a random string of the same length for comparison purposes. We do this so we can check whether or not the message is random noise or has a structure.

In [ ]:
import random

In [ ]:
rand = random.choices([0,1], k=len(text)) # Generates a random selection of 0's and 1's up to length of the message text
rand = [str(i) for i in rand] # Turns the selection into string data
rand = "".join(rand)

In [ ]:
text_d = [text.count('0')/len(text), text.count('1')/len(text)] # Gets the probability of selecting a 0 or a 1 in the distribution
rand_d = [rand.count('0')/len(rand), rand.count('1')/len(rand)]

In [ ]:
ent_t = sp.stats.entropy(text_d, base = 2)
ent_r = sp.stats.entropy(rand_d, base = 2)

messages = pd.DataFrame()
messages['type'] = ['message', 'random']
messages['entropy'] = [ent_t, ent_r]

sns.pointplot(x = 'type', y = 'entropy', data = messages)

#print(ent_t, ent_r)

## Identifying the alien alphabet

This tells is that the message is more predictable than a random string of 0's and 1's, but not much else. Let's make a few assumptions and see how far we can go.

* The aliens have a finite number of symbols that are analogous to the letters of the alphabet (i.e. they won't encode words directly)
* Each symbol will be encoded by groups of 0's and 1's
* The message will try to balance ease of decoding against expressive power
* The message is complete
* There will be repeated symbols in the message
* There will be a distinct (and obvious) character for a space between words

The simplest possible symbolic system has only two states:

$$(0) = S_1$$
$$(1) = S_2$$

Now, let's imagine a symbolic system where every symbol is associated with two binary digits. This gives us four possible symbols:

$$(0,0) = S_1$$
$$(0,1) = S_2$$
$$(1,0) = S_3$$
$$(1,1) = S_4$$

Notice that though the length of the encoding is always the same (i.e. 2), we can get four symbols. 

But are four symbols enough for an expressive alphabet? Not really. If we encode using three binary digits, we get eight ($2^3$) symbols, which is still too small. But if we use (say) ten digits, we get 1,024 ($2^{10}$) symbols which is likely too big. What's the most likely choice for the aliens to make? One clue comes from the length of the message. If it's complete, then the length of the encoding *must* evenly divide the message length. Our message is 288 digits long.

This gives the following options for the length of the encoding:

In [ ]:
symbol_length_t = sympy.divisors(len(text)) # Gets the numbers that evenly divide the message length using the `sympy` library for algebra

In [ ]:
symbol_length_t

Now that we have all the possible encoding lengths, we can divide up the message into chunks matching each possible 'alphabet'. *One* of these will be the correct one––but we don't know which!

In [ ]:
alphabets_t = []

for j in symbol_length_t:
    alphabets_t.append([text[i:i+j] for i in range(0, len(text), j)]) # Chunks the message into groups of 0's and 1's corresponding to all possible encoding lengths
    

# Get a count of how often each possible symbol occurs in the message for each alphabet and convert to pandas series
alphabets_t =[pd.Series(Counter(i)) for i in alphabets_t]

#sns.lineplot(x = alphabets_t.index, y = alphabets_t.values)

### So, we need to compare our possible message to our random string so we can see where differences emerge. We therefore do the same chunking for our random string:

In [ ]:
symbol_length_r = sympy.divisors(len(rand))

alphabets_r = []

for j in symbol_length_r:
    alphabets_r.append([rand[i:i+j] for i in range(0, len(rand), j)])
    
alphabets_r = [pd.Series(Counter(i)) for i in alphabets_r]

Now, let's calculate the entropy of each possible alphabet for the message string and the random string:

In [ ]:
entropies_t = [sp.stats.entropy(i, base = 2) for i in alphabets_t] # entropies of all message alphabets
entropies_r = [sp.stats.entropy(i, base = 2) for i in alphabets_r] # entropies of all random string alphabets

The next step comes with plotting all possible encoding lengths for the both the message and the random string and comparing one against the other.

In [ ]:
# Create a panddas dataframe from the entropy data to compare the message and the random texts

entropy = pd.DataFrame()
entropy['random'] = entropies_r
entropy['message'] = entropies_t

entropy_m = pd.melt(entropy)

entropy_m.columns = ['source', 'entropy']

entropy_m['encoding_length'] = symbol_length_r + symbol_length_t

# Plot the entropy for both message texts
sns.pointplot(x = 'encoding_length', y = 'entropy', hue = 'source', data = entropy_m)


In [ ]:
entropy['difference'] = entropy['random'] - entropy['message']
entropy['encoding_length'] = symbol_length_r

In [ ]:
entropy

### What does this tell us? It tells us that each letter of our alien alphabet is encoded with 8 binary digits. But what about words?

You'll remember that one our assumptions was that there would be an obvious candidate for the space between words. What is it?

In [ ]:
alien_letters = [text[i:i+8] for i in range(0, len(text), 8)]
alien_letters_set = list(set(alien_letters))


In [ ]:
alien_letters_set

In [ ]:
alien_words = text.split('00000000')

In [ ]:
alien_words

# 2. Semantics

## Word embeddings

The last ten years have seen the study of semantics revolutionised by the use of word embeddings. These are work by locating a word in a high-dimensional space, where words with similar meanings are close to each other in this space. They are derived by training a neural network on large samples of linguistic data. It's task is to learn a mathematical representation that minmimises the distance between words that frequently occur together.



In [ ]:
concepts = ['hammer', 'chisel', 'axe', 'file', 'cat', 'dog', 'zebra', 'dinosaur', 'apple', 'pear', 'grape', 'melon', \
           'cloud', 'moon', 'sun', 'star', 'ocean', 'france', 'germany', 'australia', 'doctor', 'mechanic', 'vet', 'teacher']

concept_v = [nlp(i).vector for i in concepts]

vectors_concepts = pd.DataFrame(concept_v, index = [i for i in concepts])

pca = PCA(n_components = 3)
comps = pca.fit_transform(vectors_concepts)
pc_df = pd.DataFrame(data = comps, columns = ['PC'+str(i) for i in range(1, comps.shape[1]+1)])
pc_df['words'] = vectors_concepts.index

fig = px.scatter_3d(pc_df, x='PC1', y='PC2', z='PC3', 
              hover_data = ['words'])

fig.update_traces(marker=dict(size = 5, line=dict(width=2,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))

fig.show()



## What's our Rosetta stone?

![Rosetta](rosetta.png)

In [ ]:
alien_vectors = pd.read_pickle("alien_vectors.pkl")

In [ ]:
sample = [
    "atmosphere",
    "lights",
    "aurora",
    "existence",
    "alive",
    "thinks",
    "peace",
    "exchange",
    "celestial",
    "comet",
    "watching",
    "constellation",
    "cosmic",
    "away",
    "distance",
    "crater",
    "eclipse",
    "galaxy",
    "gravity",
    "interstellar",
    "meteor",
    "nebula",
    "orbit",
    "planet",
    "satellite",
    "solar",
    "space",
    "spacecraft",
    "stars",
    "skies",
    "telescope",
    "universe",
    "vacuum",
    "velocity",
    "zenith",
    "person"
]


In [ ]:
vectors = [nlp(i).vector for i in sample]

In [ ]:
vectors_human = pd.DataFrame(vectors, index = [i for i in sample])

In [ ]:
vectors_human['origin'] = 'human'

In [ ]:
all_vectors = pd.concat([vectors_human, vectors_alien])

In [ ]:
pca_1 = PCA(n_components = 3)
comps_1 = pca_1.fit_transform(all_vectors.drop(['origin'], axis = 1))
pc_df_1 = pd.DataFrame(data = comps_1, columns = ['PC'+str(i) for i in range(1, comps_1.shape[1]+1)])
pc_df_1['words'] = all_vectors.index
pc_df_1['origin'] = [i for i in all_vectors['origin']]

In [ ]:
pc_df_1

In [ ]:
fig = px.scatter_3d(pc_df_1, x='PC1', y='PC2', z='PC3', color = 'origin',
              hover_data = ['words'])

fig.update_traces(marker=dict(size = 5, line=dict(width=2,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))

fig.show()

# Notebook ends